In [1]:
import gzip
#import mitosheet
#mitosheet.sheet()
import scipy.stats as stats
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt

In [2]:
# Load mutant genes for CRC from Cosmic and GWAS
## Cosmic_CancerGeneCensus_v102_GRCh38.tsv

CRC_COSMIC = pd.read_csv('../data/CRC_COSMIC_mutantcensus_genes.csv')
CRC_COSMIC = np.unique(CRC_COSMIC['CRC_COSMIC_mutant_GENE_SYMBOL'])
CRC_COSMIC

GWAS_CRC_catalog_df = pd.read_csv('../data/GWAS_CRC_catalog_genes.csv')
GWAS_CRC_catalog = np.array(GWAS_CRC_catalog_df['GWAS_CRC_catalog_genes'], dtype='str')
GWAS_CRC_catalog

print(len(CRC_COSMIC))
print(len(GWAS_CRC_catalog))


744
1586


## Enrichment of GWAS and COSMIC CRC genes in DEGs 


In [3]:
# Get background genes
BG = pd.read_csv('../data/background_genes_0.01.csv', sep=',')
# make index column of BG as an array   
BG = np.array(BG['x'], dtype='str')
print(len(BG))
BG

# Total DEG genes of FAP_P vs HC
GOI1 = pd.read_csv('../data/DE_output_contrastlist_2.csv', sep=',', index_col=0)
# keep if p_adj < 0.05 and abs(logFC) > 0.25
GOI1 = GOI1[GOI1['p_adj'] < 0.05]
GOI1 = GOI1[GOI1['logFC'].abs() > 1]
GOI1 = GOI1[GOI1['contrast'] == 'FAP_Polyp-Healthy_Normal']

# Get only total unique DEGs across all clusters
Total_DEGs = GOI1['gene'].unique()
print(f'Total unique DEGs: {len(Total_DEGs)} genes')

# Define enrichment function for a single gene set
def fisher_exact_enrichment(GOI, BG, catalog, catalog_name):
    """
    Perform Fisher's exact test enrichment analysis
    
    Parameters:
    GOI (array): Gene set of interest
    BG (array): Background gene set
    catalog (list): List of catalog genes (e.g., GWAS CRC genes)
    catalog_name (str): Name of the catalog for display
    
    Returns:
    dict: Enrichment results (odds ratio, p-value)
    """
    # Convert to sets
    BG_set = set(BG)
    GWAS_set = set(catalog)
    GOI_set = set(GOI)
    
    # Calculate contingency table values
    a = len(GOI_set & GWAS_set)  # Genes in both GOI and catalog
    b = len(GOI_set - GWAS_set)  # Genes in GOI only
    c = len((BG_set - GOI_set) & GWAS_set)  # Genes in BG and catalog only
    d = len((BG_set - GOI_set) - GWAS_set)  # Genes in BG only
    
    # Perform Fisher's exact test
    odds_ratio, p_value = stats.fisher_exact([[a, b], [c, d]], alternative='greater')
    
    # Print contingency table
    print(f"\n{catalog_name} Contingency Table:")
    print(f"                 In {catalog_name}  Not in {catalog_name}")
    print(f"In DEGs          {a:10d}    {b:10d}")
    print(f"Not in DEGs      {c:10d}    {d:10d}")
    
    return {
        'Catalog': catalog_name,
        'Num_Genes': len(GOI_set),
        'Catalog_Overlap': a,
        'Odds_Ratio': odds_ratio,
        'P_Value': p_value
    }

# Perform enrichment analysis for total DEGs
print('\n' + '='*60)
print('Enrichment Analysis: Total DEGs in CRC_COSMIC')
print('='*60)
result_cosmic = fisher_exact_enrichment(Total_DEGs, BG, CRC_COSMIC, 'CRC_COSMIC')
print(f"\nResults:")
print(f"Odds Ratio: {result_cosmic['Odds_Ratio']:.4f}")
print(f"P-Value: {result_cosmic['P_Value']:.4e}")

print('\n' + '='*60)
print('Enrichment Analysis: Total DEGs in GWAS_CRC_catalog')
print('='*60)
result_gwas = fisher_exact_enrichment(Total_DEGs, BG, GWAS_CRC_catalog, 'GWAS_CRC_catalog')
print(f"\nResults:")
print(f"Odds Ratio: {result_gwas['Odds_Ratio']:.4f}")
print(f"P-Value: {result_gwas['P_Value']:.4e}")

# Combine results into a DataFrame
enrichment_results_FAPPvsHC = pd.DataFrame([result_cosmic, result_gwas])
print('\n' + '='*60)
print('Summary of Enrichment Results')
print('='*60)
print(enrichment_results_FAPPvsHC)

11639
Total unique DEGs: 4660 genes

Enrichment Analysis: Total DEGs in CRC_COSMIC

CRC_COSMIC Contingency Table:
                 In CRC_COSMIC  Not in CRC_COSMIC
In DEGs                 262          4398
Not in DEGs             324          7322

Results:
Odds Ratio: 1.3463
P-Value: 3.0621e-04

Enrichment Analysis: Total DEGs in GWAS_CRC_catalog

GWAS_CRC_catalog Contingency Table:
                 In GWAS_CRC_catalog  Not in GWAS_CRC_catalog
In DEGs                 370          4290
Not in DEGs             364          7282

Results:
Odds Ratio: 1.7254
P-Value: 7.0634e-13

Summary of Enrichment Results
            Catalog  Num_Genes  Catalog_Overlap  Odds_Ratio       P_Value
0        CRC_COSMIC       4660              262    1.346266  3.062137e-04
1  GWAS_CRC_catalog       4660              370    1.725416  7.063441e-13
